In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
import numpy as np
 
#Load Data with pandas, and parse the first column into datetime
data=pd.read_csv('data/alldistricts.csv')
data.head()

/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
data.drop(data.columns[[0]], axis=1, inplace=True)
data.head()

In [ ]:
data.rename(columns={"CCN": "CRIME-COUNT"}, inplace=True)

In [ ]:
data['bins'] = np.nan

In [ ]:
data.head()

In [ ]:
data['CRIME-COUNT'].describe()

In [ ]:
data.loc[data['CRIME-COUNT']<=62, ['bins']] = 'VeryLow'


In [ ]:
data.loc[(data['CRIME-COUNT']>=63) & (data['CRIME-COUNT']<=83), ['bins']] = 'Low'

In [ ]:
data.loc[(data['CRIME-COUNT']>=84) & (data['CRIME-COUNT']<=104), ['bins']] = 'Medium'

In [ ]:
data.loc[(data['CRIME-COUNT']>=105) & (data['CRIME-COUNT']<=125), ['bins']] = 'High'

In [ ]:
data.loc[(data['CRIME-COUNT']>=126, ['bins'])] = 'VeryHigh'

In [ ]:
# data.loc[data['CRIME-COUNT']<=80, ['bins']] = 'Low'

In [ ]:
# data.loc[(data['CRIME-COUNT']>=81) & (data['CRIME-COUNT']<=100), ['bins']] = 'Medium'

In [ ]:
# data.loc[data['CRIME-COUNT']>=101, ['bins']] = 'High'

In [ ]:
data['bins'].value_counts()

In [ ]:
data.isnull().any()

In [ ]:
data.head()

In [ ]:
X = data.drop(["bins", "DATE", "STATION", "NAME", "CRIME-COUNT", "Year"], axis=1)
y = data["bins"]

In [ ]:
data.head()

In [ ]:
# X['TMAX'] = X['TMAX'].astype(float)
# X['TMIN'] = X['TMIN'].astype(float)
# X['Month'] = X['Month'].astype(float)
# X['Date'] = X['Date'].astype(float)
# X['Day'] = X['Illum'].astype(float)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [ ]:
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=10))
model.add(Dropout(.1))
model.add(Dense(units=100, activation='sigmoid'))
model.add(Dense(units=100, activation='tanh'))
model.add(Dropout(.1))
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(.1))
model.add(Dense(units=100, activation='tanh'))
model.add(Dropout(.1))
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(.1))
model.add(Dense(units=5, activation='softmax'))

In [ ]:
model.compile(optimizer='nadam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

In [ ]:
# X_train_scaled

In [ ]:
# y_train_categorical.shape

In [ ]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
encoded_predictions = model.predict_classes(X_test_scaled[:30])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [ ]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:30])}")

In [ ]:
model.save("static/models/alldistricts.h5")

In [ ]:
model.save_weights("static/models/alldistricts_weights.h5")

In [ ]:
with open('static/models/model_architecture_alldistricts.json', 'w') as f:
    f.write(model.to_json())

In [ ]:
# from tensorflow.keras.models import load_model
# district1_model = load_model("static/models/district1.h5")

In [ ]:
# district1_model.save_weights("static/models/district1_weights.h5")
# with open('static/models/model_architecture_district1.json', 'w') as f:
#     f.write(district1_model.to_json())

In [ ]:
# district2_model = load_model("static/models/district2.h5")
# district2_model.save_weights("static/models/district2_weights.h5")
# with open('static/models/model_architecture_district2.json', 'w') as f:
#     f.write(district2_model.to_json())

In [ ]:
# district3_model = load_model("static/models/district3.h5")
# district3_model.save_weights("static/models/district3_weights.h5")
# with open('static/models/model_architecture_district3.json', 'w') as f:
#     f.write(district3_model.to_json())

In [ ]:
# district4_model = load_model("static/models/district4.h5")
# district4_model.save_weights("static/models/district4_weights.h5")
# with open('static/models/model_architecture_district4.json', 'w') as f:
#     f.write(district4_model.to_json())

In [ ]:
# district5_model = load_model("static/models/district5.h5")
# district5_model.save_weights("static/models/district5_weights.h5")
# with open('static/models/model_architecture_district5.json', 'w') as f:
#     f.write(district5_model.to_json())

In [ ]:
# district6_model = load_model("static/models/district6.h5")
# district6_model.save_weights("static/models/district6_weights.h5")
# with open('static/models/model_architecture_district6.json', 'w') as f:
#     f.write(district6_model.to_json())

In [ ]:
# district7_model = load_model("static/models/district7.h5")
# district7_model.save_weights("static/models/district7_weights.h5")
# with open('static/models/model_architecture_district7.json', 'w') as f:
#     f.write(district7_model.to_json())